# 6.12.1. Función de tensión de Love (en coordenadas axisimétricas)

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Marzo 24, 2023      | Adaptando el código de MAXIMA a PYTHON |

## Cálculo de los esfuerzos en términos de la función de tensión de Love

In [1]:
from sympy import init_printing, latex, sin, cos, symbols, diff, Function, Eq, solve, trigsimp, expand, simplify

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Se definen las variables y funciones simbólicas necesarias:

In [2]:
r, z, E, nu = symbols('r, z, E, nu')
phi         = Function('phi')(r, z)    # función de tensión de Love

Se define el laplaciano en coordenadas axisimétricas:
\begin{equation}
\nabla^2\phi = \frac{1}{r}\frac{\partial}{\partial r}\left(r\frac{\partial\phi}{\partial r}\right) + \frac{\partial^2\phi}{\partial z^2} = \frac{\partial^2 \phi}{\partial r^2} + \frac{1}{r}\frac{\partial \phi}{\partial r} + \frac{\partial^2\phi}{\partial z^2}
\end{equation}

In [3]:
lapl = lambda f : (1/r)*diff(r*diff(f,r),r) + diff(f,z,2)

Se calcula el biarmónico $\nabla^4\phi$ de la función de tensión de Love:

In [4]:
bphi = expand(lapl(lapl(phi)))

Se definen los esfuerzos utilizando la función de tensión de Love:
\begin{align}    
    \sigma_r(r,z)      &= \frac{\partial}{\partial z}\left(\nu\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial r^2}\right)\\
    \sigma_\theta(r,z) &= \frac{\partial}{\partial z}\left(\nu\nabla^2\phi(r,z) - \frac{1}{r}\frac{\partial \phi(r,z)}{\partial r}\right)\\
    \sigma_z(r,z)      &= \frac{\partial}{\partial z}\left((2-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right)\\
    \tau_{rz}(r,z)     &= \frac{\partial}{\partial r}\left((1-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right)
\end{align}

In [5]:
sr  = diff(nu*lapl(phi) - diff(phi,r,2),     z)
st  = diff(nu*lapl(phi) - (1/r)*diff(phi,r), z)
sz  = diff((2-nu)*lapl(phi) - diff(phi,z,2), z)
trz = diff((1-nu)*lapl(phi) - diff(phi,z,2), r)

Se definen los desplazamientos utilizando la función de tensión de Love:
\begin{align}
u_r(r,z) &= -\frac{1+\nu}{E}\frac{\partial^2 \phi(r,z)}{\partial r \partial z}\\
w(r,z) &= \frac{1+\nu}{E}\left[2(1-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right].
\end{align}

In [6]:
ur = -((1+nu)/E)*diff(phi, r,1, z,1)
w  =  ((1+nu)/E)*(2*(1-nu)*lapl(phi) - diff(phi,z,2))

Se reemplazan los esfuerzos en las ecuaciones de equilibrio, para el caso axisimétrico, haciendo la fuerza másica constante ($b_r = b_\theta = 0$):
\begin{align}
   \frac{\partial \sigma_r}{\partial r} +  \frac{\partial \tau_{rz}}{\partial z} + \frac{\sigma_r - \sigma_\theta}{r} + b_r &= 0 \\
   \frac{\partial \tau_{rz}}{\partial r} + \frac{\partial \sigma_z}{\partial z} + \frac{\tau_{rz}}{r} + b_z &= 0
\end{align}

In [7]:
# Primera ecuación de equilibrio:
eq1 = expand(diff(sr,r) + diff(trz,z) + (sr - st)/r) # == 0
imprimir("", eq1, " = 0")

<IPython.core.display.Math object>

In [8]:
# Segunda ecuación de equilibrio:
eq2 = diff(trz,r) + diff(sz,z) + trz/r # == 0
imprimir("", expand(eq2 - (1-nu)*bphi), " = 0")

<IPython.core.display.Math object>

De aquí se deduce que la primera ecuación de equilibrio desaparece y la segunda, se convierte en:
\begin{equation}
(1 - \nu)\nabla^4\phi(r,z) = 0
\end{equation}

Se reemplazan los esfuerzos en las ecuaciones de compatibilidad:

In [9]:
Theta = sr + st + sz
co1 = lapl(sr) - (2/r**2)*(sr - st) + (1/(1+nu))*diff(Theta,r,2)
co2 = lapl(st) + (2/r**2)*(sr - st) + (1/(1+nu))*(1/r)*diff(Theta,r)
co3 = lapl(sz) + (1/(1+nu))*diff(Theta,z,2)
co4 = lapl(trz) - (1/r**2)*trz + (1/(1+nu))*diff(Theta,r,1,z,1)

Se verifica la equivalencia de las anteriores expresiones:

In [10]:
imprimir("", simplify(co1 - nu*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [11]:
imprimir("", simplify(co2 - nu*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [12]:
imprimir("", simplify(co3 - (2-nu)*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [13]:
imprimir("", simplify(co4 - (1-nu)*diff(bphi,r)), " = 0")

<IPython.core.display.Math object>

Se definen las primeras derivadas utilizando la regla de la cadena:

In [14]:
d_dx = lambda f: diff(f,r)*cos(t) - diff(f,t)*sin(t)/r
d_dy = lambda f: diff(f,r)*sin(t) + diff(f,t)*cos(t)/r
d_dz = lambda f: diff(f,z)

Se especifican los esfuerzos $\sigma_x$, $\sigma_y$ y $\tau_{xy}$ en función de $\sigma_r$, $\sigma_\theta$ y $\tau_{r\theta}$:
\begin{align}
\sigma_x  &= \sigma_r\cos^2\theta + \sigma_\theta\sin^2\theta - \tau_{r\theta}\sin 2\theta \\
\sigma_y  &= \sigma_r\sin^2\theta + \sigma_\theta\cos^2\theta + \tau_{r\theta}\sin 2\theta \\
\tau_{xy} &= \frac{1}{2}\left(\sigma_r - \sigma_\theta\right)\sin 2\theta + \tau_{r\theta}\cos 2\theta
\end{align}

In [15]:
sx  = sr*cos(t)**2 + st*sin(t)**2 - trt*sin(2*t)
sy  = sr*sin(t)**2 + st*cos(t)**2 + trt*sin(2*t)
txy = (sr - st)*sin(2*t)/2  + trt*cos(2*t)

NameError: name 't' is not defined

Relaciones de Airy para los esfuerzos, ecuación (***) cuando no se tienen fuerzas másicas, esto es haciendo $V(x,y)=0$:
\begin{align}
\sigma_x(x,y)  & = \frac{\partial^2 \phi(x,y)}{\partial y^2} + V(x,y) \\
\sigma_y(x,y)  & = \frac{\partial^2 \phi(x,y)}{\partial x^2} + V(x,y) \\    
\tau_{xy}(x,y) & = -\frac{\partial^2 \phi(x,y)}{\partial x \partial y}.
\end{align}

In [ ]:
eq1 = Eq(sx,   d_dy(d_dy(phi)))
eq2 = Eq(sy,   d_dx(d_dx(phi)))
eq3 = Eq(txy, -d_dx(d_dy(phi)))

Se calculan $\sigma_r$, $\sigma_\theta$ y $\tau_{r\theta}$ en términos de la función de tensión de Airy $\phi$:

In [ ]:
sol = trigsimp(solve([eq1, eq2, eq3], [sr, st, trt]))
sr  = sol[sr].expand()
st  = sol[st]
trt = sol[trt].expand()

In [ ]:
imprimir(r"\sigma_r(r, \theta) = ", sr)

In [ ]:
imprimir(r"\sigma_\theta(r, \theta) = ", st)

In [ ]:
imprimir(r"\tau_{r\theta}(r, \theta) = ", trt)

## Verificación de que la función de tensión de Airy satisface las ecuaciones de equilibrio

Se reemplaza en las ecuaciones diferenciales de equilibrio
\begin{align}
 \frac{\partial \sigma_r}{\partial r} + \frac{1}{r}\frac{\partial \tau_{r\theta}}{\partial \theta} + \frac{\sigma_r - \sigma_\theta}{r} + b_r &= 0 \\
 \frac{\partial \tau_{r\theta}}{\partial r} + \frac{1}{r}\frac{\partial \sigma_\theta}{\partial \theta}  + \frac{2 \tau_{r\theta}}{r} + b_\theta &= 0
\end{align}
asumiendo que las fuerzas másicas son nulas, esto es $b_r = b_\theta = 0$:

In [ ]:
# Primera ecuación de equilibrio:
eq1 = expand(diff(sr,r) + (1/r)*diff(trt,t) + (sr - st)/r) # == 0
imprimir("", eq1, " = 0")

In [ ]:
# Segunda ecuación de equilibrio:
eq2 = expand(diff(trt,r) + (1/r)*diff(st,t) + 2*trt/r) # == 0
imprimir("", eq2, " = 0")

Se reemplaza en la ecuación diferencial bidimensional de compatibilidad en términos de esfuerzos
\begin{equation}
\left(\frac{\partial^2}{\partial r^2} + \frac{1}{r}\frac{\partial}{\partial r} + \frac{1}{r^2}\frac{\partial^2}{\partial \theta^2}\right)\left(\sigma_r + \sigma_\theta\right) =K_1 \left(\frac{\partial b_r}{\partial r} + \frac{b_r}{r} + \frac{1}{r}\frac{\partial b_\theta}{\partial \theta}\right)
\end{equation}
asumiendo que las fuerzas másicas son nulas, esto es $b_r = b_\theta = 0$:

In [ ]:
eq3 = expand(lapl(sr + st)) # == 0
imprimir("", eq3, " = 0")

Se calcula el biarmónico de la función de tensión de Airy en coordenadas polares:

In [ ]:
biarm = expand(lapl(lapl(phi)))

imprimir(r"\nabla^4 \phi(r, \theta) = ", biarm)

Observe que la ecuación diferencial bidimensional de compatibilidad en términos de esfuerzos se reduce al biarmónico de $\phi$:

In [ ]:
eq3 - biarm

In [ ]:
/* Las constantes de Lamé: */
lamda : E*nu/((1 + nu)*(1 - 2*nu))$
G     : E/(2*(1 + nu))$

/* Dilatación cúbica para el caso axisimétrico, ecuación (*\eqref{eq:e_coord_cil}*) */
e : diff(ur,r) + ur/r + diff(w,z)$

/* Se reemplazan los desplazamientos en las ecuaciones de Cauchy- */
/* Navier (*\eqref{eq:CN_axisimetrico}*) asumiendo que las fuerzas másicas son nulas: br = bz = 0 */
cn1 : (lamda + G)*diff(e,r) + G*(lapl(ur) - ur/r^2)$
cn2 : (lamda + G)*diff(e,z) + G*lapl(w)$
factor(cn1 - 0);                      /* Imprime 0 */
factor(cn2 - (1-nu)*bphi);            /* Imprime 0 */